<a href="https://colab.research.google.com/github/Saim-Hassan786/Learn-Agentic-AI-With-OpenAI-Agents-SDK/blob/main/04-Tools/Tools.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Tools
Tools are additional features of an Agent that let the Agent take actions like retrieving data , calling external APIs, web searching, controlling computer, image generating and many more.There are 3 types of Tools :

1. **Hosted Tools**: That are hosted on the LLM Servers like code file search, web search , computer use etc.
2. **Python functions** : can also be converted as tools to integrate in our Agent.
3. **Agent.as_tools** : Agents can also be converted and used as tools.

In [ ]:
# Installing the SDK
!pip install -Uq openai-agents

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.1/40.1 kB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.6/130.6 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.3/129.3 kB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 150.9/150.9 kB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 kB 1.9 MB/s eta 0:00:00


In [ ]:
# For running event loop
import nest_asyncio
nest_asyncio.apply()

In [ ]:
# Pre requisites SetUp
from google.colab import userdata
GOOGLE_API_KEY= userdata.get('GOOGLE_API_KEY')

from agents import set_default_openai_api,set_default_openai_client,set_tracing_disabled
from openai import AsyncOpenAI

external_client = AsyncOpenAI(
    base_url = "https://generativelanguage.googleapis.com/v1beta/openai/",
    api_key = GOOGLE_API_KEY
)
set_default_openai_client(external_client)
set_default_openai_api("chat_completions")
set_tracing_disabled(True)

#  Tools Execution

In [ ]:
# Case No.1  === Simple Agent_With_Tool Setup
from agents import Agent, Runner, function_tool

@function_tool
def add(a: int, b: int) -> int:
    """Add two integers together"""
    return a + b

@function_tool
def multiply(a: int, b: int) -> int:
    """Multiply two integers together"""
    return a * b

agent = Agent(
    name="Assistant Agent",
    instructions = "Help With the user queries",
    model = "gemini-2.5-flash",
    tools = [add,multiply]
)

result = await Runner.run(
    agent,
    input = "What is the answer of 2+2?"
)
print(result.final_output)

The answer to 2+2 is 4.


In [ ]:
# Case No.2  === Tools when called no matter what they return that return is always sent to LLM for further processing
from agents import Agent, Runner, function_tool

@function_tool
def add_2(a: int, b: int) :
    """Add two integers together"""
    return None

@function_tool
def multiply_2(a: int, b: int):
    """Multiply two integers together"""
    return "Do not know the Answer"


agent_2 = Agent(
    name="Assistant Agent",
    instructions = "Help With the user queries",
    model = "gemini-2.5-flash",
    tools = [add_2,multiply_2]
)

result_2 = await Runner.run(
    agent_2,
    input = "What is the answer of 20 multiply by 2 using 'multiply' tool ?"
)
print(result_2.final_output)

result_3 = await Runner.run(
    agent_2,
    input = "What is the answer of adding 100 and 100 using 'add' tool ?"
)

print(result_3.final_output)

I'm sorry, I was unable to find the answer to 20 multiplied by 2 using the 'multiply_2' tool.
The tool did not return an answer.


# Other Ways Of Tool Creation

In [ ]:
from pydantic import BaseModel
from agents import Agent, Runner, FunctionTool,RunContextWrapper

class FunctionArgs(BaseModel):
  username : str
  age : int

async def on_invoke_func(ctx:RunContextWrapper,args):
  parsed = FunctionArgs.model_validate_json(args)
  return f"Hello {parsed.username} you are {parsed.age} years old 🚀"

display_user_info_tool = FunctionTool(
    name = "UserInfo_Disply_Tool",
    description= "Display Username and Age",
    on_invoke_tool=on_invoke_func,
    params_json_schema=FunctionArgs.model_json_schema(),
    is_enabled=True,
    strict_json_schema=True
)

agent_with_FunctionTool = Agent(
    name="Assistant Agent",
    instructions = "Help With the user queries",
    model = "gemini-2.5-flash",
    tools = [display_user_info_tool]

)

result_with_FunctionTool = await Runner.run(
    agent_with_FunctionTool,
    input = "My name is Saim Hassan and my age is 25 years , display my info"
)
print(result_with_FunctionTool.final_output)

Hello Saim Hassan you are 25 years old 🚀



In [ ]:
from pydantic import BaseModel
from agents import Agent, Runner, FunctionTool,RunContextWrapper

class DivisionFuncArgs(BaseModel):
  a : int
  b : int

async def on_invoke_func_div(ctx:RunContextWrapper,args):
  parsed = DivisionFuncArgs.model_validate_json(args)
  return f"Hurrah 😊 {parsed.a / parsed.b}"

division_tool = FunctionTool(
    name = "Division_Tool",
    description = "Divide two numbers",
    on_invoke_tool = on_invoke_func_div,
    params_json_schema = DivisionFuncArgs.model_json_schema(),
    is_enabled = True,
    strict_json_schema = True
)

agent_with_FunctionTool_2 = Agent(
    name="Assistant Agent",
    instructions = "Help With the user queries",
    model = "gemini-2.5-pro",
    tools = [division_tool],
    tool_use_behavior="stop_on_first_tool"

)

result_with_FunctionTool_2 = await Runner.run(
    agent_with_FunctionTool_2,
    input = "Divide 100 by 5 using the division_tool and give the answer"
)
print(result_with_FunctionTool_2.final_output)

Hurrah 😊 20.0


# @function_tool decorator parameters

In [ ]:
from agents import Agent, Runner, function_tool

def error_func(ctx:RunContextWrapper,error: Exception) -> str:
    print(f"Error Occured 🎃 {error}")
    return f"Error occurred: {str(error)}. Please try again."

@function_tool(
    name_override="Addition_tool",
    description_override="Add two integers together",
    docstring_style=None,
    use_docstring_info=True,
    strict_mode=True,
    is_enabled=True,
    failure_error_function=error_func
)
def add_with_function_tool_params(a: int, b: int) :
    """Add two integers together"""
    return a+b


@function_tool(
    name_override="Division_tool",
    description_override="Divide two integers",
    docstring_style=None,
    use_docstring_info=True,
    strict_mode=True,
    is_enabled=True,
    failure_error_function=error_func
    # failure_error_function=None  In case of None, the Error does not goes to LLM , instead it raises above in the Loop and Terminates it.
)
def division_with_function_tool_params(a: int, b: int):
    """Divide two integers"""
    return a/b


agent_with_function_tool_params = Agent(
    name="Assistant Agent",
    instructions = "Help With the user queries",
    model = "gemini-2.5-flash",
    tools = [add_with_function_tool_params,division_with_function_tool_params],
    # tool_use_behavior="stop_on_first_tool"
)

result_with_function_tool_params = await Runner.run(
    agent_with_function_tool_params,
    input = "What is the answer of 115 by 0 using the division tool ?"
)
print(result_with_function_tool_params.final_output)

Error Occured 🎃 division by zero
I cannot divide by zero. Please provide a non-zero number for division.


# Conclusion
Tools can be created by 2 ways :

1. **@function_tool**
2. **FunctionTool**

Both of all return :

  - **FuntionToolResult** containing
      
      - **tool** : FunctionTool
      - **output**
      - **run_items**

